##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Convierta su código existente a TensorFlow 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/migration_guide">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    Ver en TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/migration_guide.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Correr en Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/migration_guide.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    Ver codigo on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/migration_guide.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Descargar notebook</a>
  </td>
</table>


Note: Nuestra comunidad de Tensorflow ha traducido estos documentos. Como las traducciones de la comunidad son basados en el "mejor esfuerzo", no hay ninguna garantia que esta sea un reflejo preciso y actual de la Documentacion Oficial en Ingles. Si tienen sugerencias sobre como mejorar esta traduccion, por favor envian un "Pull request" al siguiente repositorio tensorflow/docs. Para ofrecerse como voluntario o hacer revision de las traducciones de la Comunidad por favor contacten al siguiente grupo docs@tensorflow.org list

Importante: este documento es para usuarios de bajo nivel de la API de TensorFlow . Si esta utilizando la API de alto nivel (`tf.keras`) puede que haya poca o ninguna acción que deba tomar para que su código sea totalmente compatible con TensorFlow 2.0. Verifique su [tasa de aprendizaje predeterminada del optimizer](keras_optimizer_lr).

Es posible ejecutar código 1.X sin modificar ([excepto para contrib](https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md)), en TensorFlow 2.0 :

```
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
```

Sin embargo, esto no le permite aprovechar muchas de las mejoras realizadas en TensorFlow 2.0. Esta guia lo ayudara a actualizar su codigo, haciendolo mas simple, mas eficiente y mas facil de mantener.

## Script de conversion automatica

El primer paso, antes de intentar implementar los cambios descritos en este documento, es intentar ejecutar el [script de actualización](./upgrade.md).

Esto hara un paso inicial al actualizar su codigo a TensorFlow 2.0. Pero no puede hacer que su codigo sea idiomatico para 2.0. Su codigo aun puede utilizar los endpoints de `tf.compat.v1` para acceder a marcadores de posicion, sesiones, colecciones y otras funciones de estilo 1.x

## Cambios de comportamiento de nivel superior

Si su codigo funciona en TensorFlow 2.0 usando `tf.compat.v1.disable_v2_behavior()`, todavia hay cambios de comportamiento globales que puede necesitar modificar. Los principales cambios son:

* *Eager execution, `v1.enable_eager_execution()`*: Cualquier codigo que implicitamente use un `tf.Graph` fallara. Asegurese de ajustar este codigo en un contexto `with tf.Graph(). as_default()`.
    
* *Variables de recursos, `v1.enable_resource_variables()`*: Algunos codigos pueden depender de comportamientos no deterministas habilitados por las variables de referencia TF.
Las variables de recursos se bloquean durante la escritura y, por lo tanto, proporcionan garantias de coherencia mas intuitivas.

  * Puede cambiar el comportamiento en casos extremos.
  * Puede crear copias adicionales y puede tener un mayor uso de memoria.
  * Se puede deshabilitar pasando `use_resource = False` al constructor` tf.Variable`.

* *Tensor Shapes, `v1.enable_v2_tensorshape()`* : TF 2.0 simplifica el comportamiento de las Tensor Shapes. En lugar de `t.shape[0].value` puede usar ` t.shape[0]`. Estos cambios deben ser pequeños y tiene sentido solucionarlos de inmediato. Consulte [TensorShape](#tensorshape) para ver ejemplos.

* *Control de Flujo, `v1.enable_control_flow_v2()`*: La implementacion del control de flujo TF 2.0 se ha simplificado y, por lo tanto, produce diferentes representaciones graficas. Por favor, [reporte bugs](https://github.com/tensorflow/tensorflow/issues) si se presenta cualquier problema.

## Hacer el codigo 2.0-nativo


Esta guia le mostrara varios ejemplos de conversión de codigo TensorFlow 1.x a TensorFlow 2.0. Estos cambios permitiran que su codigo aproveche las optimizaciones de rendimiento y las llamadas API simplificadas.

En cada caso, el patron es:

### 1. Remplaza las llamadas `v1.Session.run`


Cada llamada `v1.Session.run` debe ser reemplazada por una funcion Python.

* Los `feed_dict` y` v1.placeholder`s se convierten en argumentos de funcion.
* Los 'fetches' se convierten en el valor de retorno de la funcion.
* Durante la conversion, la eager execution permite una depuracion facil con herramientas estandar de Python como `pdb`.

Despues de eso, agrega un decorador `tf.function` para que funcione de manera eficiente en el grafico. Consulta la [Guía de autografos](autograph.ipynb) para obtener mas informacion sobre como funciona.

Ten en cuenta que:

* A diferencia de `v1.Session.run`, una` tf.function` tiene una firma de devolucion fija y siempre devuelve todas las salidas. Si esto causa problemas de rendimiento, crea dos funciones separadas.

* No hay necesidad de un `tf.control_dependencies` u operaciones similares: una` tf.function` se comporta como si se ejecutara en el orden escrito. Las asignaciones `tf.Variable` y` tf.assert`s, por ejemplo, se ejecutan automaticamente.

### 2. Utiliza objetos Python para rastrear variables y perdidas

Todo el seguimiento de variables basado en nombres se desaconseja en TF 2.0. Utiliza objetos Python para rastrear variables.

Utiliza `tf.Variable` en lugar de `v1.get_variable`.

Cada `v1.variable_scope` debe convertirse en un objeto Python. Por lo general, este sera uno de la siguiente lista:

* `tf.keras.layers.Layer`
* `tf.keras.Model`
* `tf.Module`

Si necesitas agregar listas de variables (como `tf.Graph.get_collection(tf.GraphKeys.VARIABLES)`), usa los atributos `.variables` y`.trainable_variables` de los objetos `Layer` y` Model`.

Las clases `Layer` y` Model` implementan varias propiedades que eliminan la necesidad de colecciones globales. Tu propiedad `.losses` puede ser un reemplazo para usar la coleccion `tf.GraphKeys.LOSSES`.

Consulta las [guías de keras](keras.ipynb) para obtener mas detalles.

Advertencia: Muchos simbolos `tf.compat.v1` usan las colecciones globales implicitamente.

### 3. Actualiza tus ciclos de entrenamiento

Usa la API de nivel mas alto que funcione para tu caso. Prefiere `tf.keras.Model.fit` en lugar de construir tus propios ciclos de entrenamiento.

Estas funciones de alto nivel manejan muchos de los detalles de bajo nivel que podrian ser faciles de perder si escribes tu propio ciclo de entrenamiento. Por ejemplo, recopilan automaticamente las perdidas de regularizacion y establecen el argumento `training = True` cuando llaman al modelo.


### 4. Actualiza tus fuentes de entrada de datos

Utiliza los datasets `tf.data`  para la entrada de datos. Estos objetos son eficientes, expresivos y se integran bien con tensorflow.

Se pueden pasar directamente al metodo `tf.keras.Model.fit`.


```
model.fit(dataset, epochs=5)
```

Se pueden iterar directamente sobre Python estandar:

```
for example_batch, label_batch in dataset:
    break
```


#### 5. Migra los simbolos de `compat.v1` 

El modulo `tf.compat.v1` contiene la API completa de TensorFlow 1.x, con su semantica original.

El [script de actualización TF2](upgrade.ipynb) convertira simbolos a sus equivalentes 2.0 si dicha conversion es segura, es decir, si puede determinar que el comportamiento de la versión 2.0 es exactamente equivalente (por ejemplo, cambiara el nombre de 'v1 .arg_max` a `tf.argmax`, ya que son la misma funcion).

Después de que el script de actualizacion se ejecute en un fragmento de codigo, es probable que haya muchas menciones de `compat.v1`. Vale la pena revisar el codigo y convertirlos manualmente al equivalente 2.0 (deberia mencionarse en el registro si hay uno).

## Convirtiendo modelos

### Instalacion

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version solo existe en Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf


import tensorflow_datasets as tfds

### Variables de bajo nivel y ejecucion del operador

Ejemplos del uso de la API de bajo nivel incluyen:

* Usando ambitos de variables para controlar la reutilizacion
* Creando variables con `v1.get_variable`.
* Accediendo a colecciones explicitamente
* Accediendo a colecciones implicitamente con metodos como:

   * `v1.global_variables`
   * `v1.losses.get_regularization_loss`

* Usando `v1.placeholder` para configurar entradas de graficos
* Ejecutando de graficos con `Session.run`
* Inicializando variables manualmente


#### Antes de convertir


Asi es como estos patrones pueden verse en el codigo utilizando TensorFlow 1.x.


```python
in_a = tf.placeholder(dtype=tf.float32, shape=(2))
in_b = tf.placeholder(dtype=tf.float32, shape=(2))

def forward(x):
  with tf.variable_scope("matmul", reuse=tf.AUTO_REUSE):
    W = tf.get_variable("W", initializer=tf.ones(shape=(2,2)),
                        regularizer=tf.contrib.layers.l2_regularizer(0.04))
    b = tf.get_variable("b", initializer=tf.zeros(shape=(2)))
    return W * x + b

out_a = forward(in_a)
out_b = forward(in_b)

reg_loss = tf.losses.get_regularization_loss(scope="matmul")

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  outs = sess.run([out_a, out_b, reg_loss],
      	        feed_dict={in_a: [1, 0], in_b: [0, 1]})

```

#### Despues de convertir

En el codigo convertido:

* Las variables son objetos locales de Python.
* La funcion `forward` sigue definiendo el calculo.
* La llamada  a `Session.run` se reemplaza con una llamada a la funcion `forward`
* El decorador opcional `tf.function` se puede agregar para aumentar el rendimiento.
* Las regularizaciones se calculan manualmente, sin hacer referencia a ninguna coleccion global.
* **No hay sesiones ni placeholders**.

In [0]:
W = tf.Variable(tf.ones(shape=(2,2)), name="W")
b = tf.Variable(tf.zeros(shape=(2)), name="b")

@tf.function
def forward(x):
  return W * x + b

out_a = forward([1,0])
print(out_a)

In [0]:
out_b = forward([0,1])

regularizer = tf.keras.regularizers.l2(0.04)
reg_loss = regularizer(W)

### Modelos basados en `tf.layers`


El modulo `v1.layers` se utiliza para contener funciones de capa que se basan en `v1.variable_scope` para definir y reutilizar variables.

#### Antes de convertir
```python
def model(x, training, scope='model'):
  with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
    x = tf.layers.conv2d(x, 32, 3, activation=tf.nn.relu,
          kernel_regularizer=tf.contrib.layers.l2_regularizer(0.04))
    x = tf.layers.max_pooling2d(x, (2, 2), 1)
    x = tf.layers.flatten(x)
    x = tf.layers.dropout(x, 0.1, training=training)
    x = tf.layers.dense(x, 64, activation=tf.nn.relu)
    x = tf.layers.batch_normalization(x, training=training)
    x = tf.layers.dense(x, 10, activation=tf.nn.softmax)
    return x

train_out = model(train_data, training=True)
test_out = model(test_data, training=False)
```

#### Despues de convertir

* La pila simple de capas encaja perfectamente en `tf.keras.Sequential`. (Para modelos más complejos, consulte [capas y modelos personalizados](keras/custom_layers_and_models.ipynb) y [la API funcional](keras/functional.ipynb)).
* El modelo rastrea las variables y las perdidas de regularizacion.
* La conversion fue uno a uno porque hay una asignacion directa de `v1.layers` a` tf.keras.layers`.

La mayoria de los argumentos permanecieron igual. Pero note las diferencias:

* El argumento `training` se pasa a cada capa por el modelo cuando se ejecuta.
* El primer argumento de la funcion original `model` (la entrada` x`) desaparecio. Esto se debe a que las capas de objetos separan la construccion del modelo de la llamada del modelo.


Tambien ten en cuenta que:

* Si estabas utilizando regularizadores de inicializadores de `tf.contrib`, estos tienen mas cambios de argumento que otros.
* El codigo ya no se escribe en las colecciones, por lo que funciones como `v1.losses.get_regularization_loss` ya no devolveran estos valores, lo que podria romper sus ciclos de entrenamiento.

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.04),
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])

train_data = tf.ones(shape=(1, 28, 28, 1))
test_data = tf.ones(shape=(1, 28, 28, 1))

In [0]:
train_out = model(train_data, training=True)
print(train_out)

In [0]:
test_out = model(test_data, training=False)
print(test_out)

In [0]:
# Aqui estan todas las variables de entrenamiento.
len(model.trainable_variables)

In [0]:
# Aqui esta la perdida de regularizacion
model.losses

### Variables combinadas & `v1.layers`


El codigo existente a menudo mezcla variables y operaciones TF 1.x de nivel inferior con `v1.layers` de nivel superior.

#### Antes de convetir
```python
def model(x, training, scope='model'):
  with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
    W = tf.get_variable(
      "W", dtype=tf.float32,
      initializer=tf.ones(shape=x.shape),
      regularizer=tf.contrib.layers.l2_regularizer(0.04),
      trainable=True)
    if training:
      x = x + W
    else:
      x = x + W * 0.5
    x = tf.layers.conv2d(x, 32, 3, activation=tf.nn.relu)
    x = tf.layers.max_pooling2d(x, (2, 2), 1)
    x = tf.layers.flatten(x)
    return x

train_out = model(train_data, training=True)
test_out = model(test_data, training=False)
```

#### Despues de convertir

Para convertir este codigo, sigue el patron de asignacion de capas a capas como en el ejemplo anterior.

Un `v1.variable_scope` es efectivamente una capa propia. Asi que reescribalo como un `tf.keras.layers.Layer`. Ver [la guia](keras/custom_layers_and_models.ipynb) para mas detalles.

El patron general es:

* Recopila parametros de capa en `__init__`.
* Construue las variables en `build`.
* Ejecuta los calculos en `call` y devuelva el resultado.

El `v1.variable_scope` es esencialmente una capa propia. Asi que reescribelo como un `tf.keras.layers.Layer`. Ver [la guia](keras/custom_layers_and_models.ipynb) para mas detalles.

In [0]:
# Crea una capa personalizada para parte del modelo.
class CustomLayer(tf.keras.layers.Layer):
  def __init__(self, *args, **kwargs):
    super(CustomLayer, self).__init__(*args, **kwargs)

  def build(self, input_shape):
    self.w = self.add_weight(
        shape=input_shape[1:],
        dtype=tf.float32,
        initializer=tf.keras.initializers.ones(),
        regularizer=tf.keras.regularizers.l2(0.02),
        trainable=True)

  # El metodo de llamada a veces se usara en modo grafico,
  # el entrenamiento se convertira en un tensor
  @tf.function
  def call(self, inputs, training=None):
    if training:
      return inputs + self.w
    else:
      return inputs + self.w * 0.5

In [0]:
custom_layer = CustomLayer()
print(custom_layer([1]).numpy())
print(custom_layer([1], training=True).numpy())

In [0]:
train_data = tf.ones(shape=(1, 28, 28, 1))
test_data = tf.ones(shape=(1, 28, 28, 1))


# Construye el modelo incluyendo la capa personalizada
model = tf.keras.Sequential([
    CustomLayer(input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
])

train_out = model(train_data, training=True)
test_out = model(test_data, training=False)


Algunas cosas a tener en cuenta:

* Los modelos y capas de Keras subclasificados deben ejecutarse tanto en graficos v1 (sin dependencias de control automatico) como en modo eager 
   * Envuelva el `call()` en un `tf.function()` para obtener dependencias de autografos y control automatico

* No olvides aceptar un argumento de 'training' para 'call'.
     * A veces es un `tf.Tensor`
     * A veces es un booleano Python.

* Crea variables modelo en constructor o `Model.build` usando` self.add_weight () `.
   * En `Model.build` tiene acceso a la forma de entrada, por lo que puede crear pesos con formas coincidentes.
   * El uso de `tf.keras.layers.Layer.add_weight` le permite a Keras rastrear variables y perdidas de regularizacion.

* No guardes `tf.Tensors` en tus objetos.
   * Pueden crearse en una `tf.function` o en un eager context, y estos tensores se comportan de manera diferente.
   * Usa `tf.Variable`s para el estado, siempre se pueden usar desde ambos contextos
   * `tf.Tensors` son solo para valores intermedios.

### Una nota sobre Slim y contrib.layers

Una gran cantidad de codigo anterior de TensorFlow 1.x utiliza la biblioteca [Slim](https://ai.googleblog.com/2016/08/tf-slim-high-level-library-to-define.html), que era empaquetado con TensorFlow 1.x como `tf.contrib.layers`. Como modulo `contrib`, esto ya no esta disponible en TensorFlow 2.0, incluso en` tf.compat.v1`. La conversion de codigo usando Slim a TF 2.0 es mas complicada que la conversion de repositorios que usan `v1.layers`. De hecho, puede tener sentido convertir su codigo Slim a `v1.layers` primero, luego convertirlo a Keras.

* Elimina `arg_scopes`, todos los argumentos deben ser explicitos
* Si los usas, divide `normalizer_fn` y` Activation_fn` en sus propias capas
* Las capas de conv separables se asignan a una o mas capas de Keras diferentes (capas de Keras separables en profundidad, pointwise y separables)
* Slim y `v1.layers` tienen diferentes nombres arg y valores predeterminados
* Algunos args tienen diferentes escalas
* Si usas modelos pre-entrenados Slim, pruebe `tf.keras.applications` o [TFHub](https://tensorflow.orb/hub)

Es posible que algunas capas `tf.contrib` no se hayan movido al nucleo de TensorFlow, sino que se hayan movido al [paquete de complementos TF](https://github.com/tensorflow/addons).


## Entrenamiento

Hay muchas formas de alimentar datos a un modelo `tf.keras`. Aceptaran generadores Python y matrices Numpy como entrada.

La forma recomendada de alimentar datos a un modelo es usar el paquete `tf.data`, que contiene una coleccion de clases de alto rendimiento para manipular datos.

Si todavia utilizas `tf.queue`, ahora solo se admiten como estructuras de datos, no como canalizaciones de entrada.

### Utilizando Datasets

El paquete [TensorFlow Datasets](https://tensorflow.org/datasets) (`tfds`) contiene utilidades para cargar conjuntos de datos predefinidos como objetos` tf.data.Dataset`.

Para este ejemplo, carga el conjunto de datos MNIST, usando `tfds`:

In [0]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

Luego prepara los datos para el entrenamiento:

   * Vuelva a escalar cada imagen.
   * Barajea el orden de los ejemplos.
   * Recoge lotes de imagenes y etiquetas.


In [0]:
BUFFER_SIZE = 10 # Utiliza un valor mas grande para codigo real
BATCH_SIZE = 64
NUM_EPOCHS = 5


def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

Para mantener el ejemplo corto, reduce el conjunto de datos para que solo devuelva 5 lotes:

In [0]:
train_data = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).take(5)
test_data = mnist_test.map(scale).batch(BATCH_SIZE).take(5)

STEPS_PER_EPOCH = 5

train_data = train_data.take(STEPS_PER_EPOCH)
test_data = test_data.take(STEPS_PER_EPOCH)

In [0]:
image_batch, label_batch = next(iter(train_data))

### Usa los ciclos de entrenamiento de Keras

Si no necesitas un control de bajo nivel de tu proceso de entrenamiento, se recomienda utilizar los metodos incorporados `fit`, `evaluate` y `predict` de Keras. Estos metodos proporcionan una interfaz uniforme para entrenar el modelo independientemente de la implementación (secuencial, funcional o subclasificada).

Las ventajas de estos metodos incluyen:

* Aceptan matrices Numpy, generadores Python y `tf.data.Datasets`
* Aplican la regularizacion, y las perdidas de activacion automaticamente.
* Admiten `tf.distribute` [para la capacitacion en multiples dispositivos](distribuir_estrategia.ipynb).
* Admiten invocaciones arbitrarias como perdidas y metricas.
* Admiten devoluciones de llamada como `tf.keras.callbacks.TensorBoard`, y devoluciones de llamada personalizadas.
* Son efectivos, automaticamente usan graficos TensorFlow.

Aqui hay un ejemplo de entrenamiento de un modelo usando un `Dataset`. (Para obtener detalles sobre como funciona, consulta [tutoriales](../tutoriales).)

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.02),
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])


# model es el modelo completo sin capas personalizadas
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=NUM_EPOCHS)
loss, acc = model.evaluate(test_data)

print("Loss {}, Accuracy {}".format(loss, acc))

### Escribe tu propio ciclo

Si el paso de entrenamiento del modelo Keras funciona para usted, pero necesita mas control, considera usar el metodo `tf.keras.Model.train_on_batch`, en su propio ciclo de iteración de datos.

Recuerda: se pueden implementar muchas cosas como un `tf.keras.callbacks.Callback`.

Este metodo tiene muchas de las ventajas de los metodos mencionados en la seccion anterior, pero le da al usuario el control del ciclo externo.

Tambien puede usar `tf.keras.Model.test_on_batch` o` tf.keras.Model.evaluate` para verificar el rendimiento durante el entrenamiento.

Nota: `train_on_batch` y` test_on_batch`, por defecto devuelven la perdida y las metricas para el lote individual. Si pasa `reset_metrics = False`, devuelven las metricas acumuladas y debe recordar restablecer adecuadamente los acumuladores de metricas. Recuerda tambien que algunas metricas como `AUC` requieren que` reset_metrics = False` se calcule correctamente.

Para continuar entrenando el modelo anterior:

In [0]:

# model es el modelo completo sin capas personalizadas
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

metrics_names = model.metrics_names

for epoch in range(NUM_EPOCHS):
  # Reinicia los acumuladores de metricas
  model.reset_metrics()

  for image_batch, label_batch in train_data:
    result = model.train_on_batch(image_batch, label_batch)
    print("train: ",
          "{}: {:.3f}".format(metrics_names[0], result[0]),
          "{}: {:.3f}".format(metrics_names[1], result[1]))
  for image_batch, label_batch in test_data:
    result = model.test_on_batch(image_batch, label_batch,
                                 # Devuelve las metricas acumuladas
                                 reset_metrics=False)
  print("\neval: ",
        "{}: {:.3f}".format(metrics_names[0], result[0]),
        "{}: {:.3f}".format(metrics_names[1], result[1]))



<a name="custom_loop"></a>

### Personaliza el paso de entrenamiento

Si necesitas mas flexibilidad y control, puedes conseguirlo implementando tu propio ciclo de entrenamiento. Hay tres pasos:

1. Itera sobre un generador de Python o `tf.data.Dataset` para obtener lotes de ejemplos.
2. Usa `tf.GradientTape` para recolectar gradientes.
3. Utiliza uno de los `tf.keras.optimizers` para aplicar actualizaciones de peso a las variables del modelo.

Recuerda:

* Incluye siempre un argumento de `training` en el metodo` call` de capas y modelos subclasificados.
* Asegurate de llamar al modelo con el argumento `training` configurado correctamente.
* Segun el uso, las variables del modelo pueden no existir hasta que el modelo se ejecute en un lote de datos.
* Necesitas manejar manualmente cosas como perdidas de regularizacion para el modelo.

Ten en cuenta las simplificaciones relativas a v1:

* No hay necesidad de ejecutar inicializadores variables. Las variables se inicializan en la creacion.
* No es necesario agregar dependencias de control manual. Incluso en las operaciones `tf.function` actúan como en el modo eager.

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.02),
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

@tf.function
def train_step(inputs, labels):
  with tf.GradientTape() as tape:
    predictions = model(inputs, training=True)
    regularization_loss = tf.math.add_n(model.losses)
    pred_loss = loss_fn(labels, predictions)
    total_loss = pred_loss + regularization_loss

  gradients = tape.gradient(total_loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

for epoch in range(NUM_EPOCHS):
  for inputs, labels in train_data:
    train_step(inputs, labels)
  print("Finished epoch", epoch)


### Nuevas metricas y perdidas de estilo

En TensorFlow 2.0, las metricas y las perdidas son objetos. Estos funcionan tanto con eagerly como en `tf.function`s.

Un objeto de perdida es invocable y espera el (y_pred, y_true) como argumentos:




In [0]:
cce = tf.losses.CategoricalCrossentropy(from_logits=True)
cce([[1, 0]], [[-1.0,3.0]]).numpy()

Un objeto de metrica tiene los siguientes metodos:

* `Metric.update_state()` - agrega nuevas observaciones
* `Metric.result ()`: obtiene el resultado actual de la metrica, dados los valores observados
* `Metric.reset_states()` - borra todas las observaciones.

El objeto en si es invocable. La llamada actualiza el estado con nuevas observaciones, como con `update_state`, y devuelve el nuevo resultado de la metrica.

No tienes que inicializar manualmente las variables de una metrica, y debido a que TensorFlow 2.0 tiene dependencias de control automatico, tampoco necesitas preocuparse por ellas.

El siguiente codigo utiliza una metrica para realizar un seguimiento de la perdida media observada dentro de un ciclo de entrenamiento personalizado.

In [0]:
# Crea las metricas
loss_metric = tf.keras.metrics.Mean(name='train_loss')
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

@tf.function
def train_step(inputs, labels):
  with tf.GradientTape() as tape:
    predictions = model(inputs, training=True)
    regularization_loss = tf.math.add_n(model.losses)
    pred_loss = loss_fn(labels, predictions)
    total_loss = pred_loss + regularization_loss

  gradients = tape.gradient(total_loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  # Actualiza las metricas
  loss_metric.update_state(total_loss)
  accuracy_metric.update_state(labels, predictions)


for epoch in range(NUM_EPOCHS):
  # Reinicia las metricas
  loss_metric.reset_states()
  accuracy_metric.reset_states()

  for inputs, labels in train_data:
    train_step(inputs, labels)
  # Obten las metricas resultantes
  mean_loss = loss_metric.result()
  mean_accuracy = accuracy_metric.result()

  print('Epoch: ', epoch)
  print('  loss:     {:.3f}'.format(mean_loss))
  print('  accuracy: {:.3f}'.format(mean_accuracy))


### Optimizadores Keras

Los optimizadores en `v1.train`, como `v1.train.AdamOptimizer` y `v1.train.GradientDescentOptimizer`, tienen equivalentes en` tf.keras.optimizers`.

#### Conviertir `v1.train` a `keras.optimizers`

Algunas cosas a tener en cuenta al convertir tus optimizadores

* Actualiza tus optimizadores [puedes hacer que los checkpoints antiguos sean incompatibles](#checkpoints).
* Todos los epsilons ahora estan predeterminados en `1e-7` en lugar de` 1e-8` (que es insignificante en la mayoria de los casos).
* `v1.train.GradientDescentOptimizer` se puede reemplazar directamente por` tf.keras.optimizers.SGD`.
* `v1.train.MomentumOptimizer` se puede reemplazar directamente por el optimizador` SGD` utilizando el argumento momentum: `tf.keras.optimizers.SGD (..., momentum = ...)`.
* `v1.train.AdamOptimizer` se puede convertir para usar` tf.keras.optimizers.Adam`. Los argumentos `beta1` y` beta2` han cambiado de nombre a `beta_1` y` beta_2`.
* `v1.train.RMSPropOptimizer` se puede convertir a` tf.keras.optimizers.RMSprop`. El argumento `decay` ha sido renombrado a` rho`.
* `v1.train.AdadeltaOptimizer` se puede convertir directamente a` tf.keras.optimizers.Adadelta`.
* `tf.train.AdagradOptimizer` se puede convertir directamente a` tf.keras.optimizers.Adagrad`.
* `tf.train.FtrlOptimizer` se puede convertir directamente a` tf.keras.optimizers.Ftrl`. Se han eliminado los argumentos `acumular_nombre` y` lineal_nombre`.
* El `tf.contrib.AdamaxOptimizer` y` tf.contrib.NadamOptimizer`, se pueden convertir directamente a `tf.keras.optimizers.Adamax` y` tf.keras.optimizers.Nadam`. Los argumentos `beta1` y` beta2` han cambiado de nombre a `beta_1` y` beta_2`.




#### Nuevos valores predeterminados para algunos `tf.keras.optimizers`
<a id="keras_optimizer_lr"></a>

Advertencia: si ves un cambio en el comportamiento de convergencia de sus modelos, verifica las tasas de aprendizaje predeterminadas.

No hay cambios para `optimizers.SGD`,` optimizers.Adam` u `optimizers.RMSprop`.

Las siguientes tasas de aprendizaje predeterminadas han cambiado:

* `optimizadores.Adagrad` de 0.01 a 0.001
* `optimizadores.Adadelta` de 1.0 a 0.001
* `optimizadores.Adamax` de 0.002 a 0.001
* `optimizadores.Nadam` de 0.002 a 0.001

### TensorBoard

TensorFlow 2.0 incluye cambios significativos en la API `tf.summary` utilizada para escribir datos de resumen para su visualización en TensorBoard. Para una introduccion general al nuevo tf.summary, hay [varios tutoriales disponibles](https://www.tensorflow.org/tensorboard/r2/get_started) que usan la API TF 2.0. Esto incluye una [Guia de migración de TensorBoard TF2.0](https://www.tensorflow.org/tensorboard/r2/migrate)

## Guardando & Cargando


<a id="checkpoints"></a>
### Compatibilidad de checkpoints

TensorFlow 2.0 utiliza [checkpoints basados ​​en objetos](checkpoints.ipynb).

Los checkpoints basados en nombres antiguos todavia se pueden cargar, si tienes cuidado.
El proceso de conversion de codigo puede dar como resultado cambios de nombre variables, pero hay soluciones alternativas.

El enfoque mas simple es alinear los nombres del nuevo modelo con los nombres en el punto de control:

* Las variables todavia tienen un argumento `name` que puede establecer.
* Los modelos Keras tambien toman un argumento `nombre` como el que establecen como prefijo para sus variables.
* La función `v1.name_scope` se puede usar para establecer prefijos de nombre de variable. Esto es muy diferente de `tf.variable_scope`. Solo afecta a los nombres, y no rastrea variables y reutiliza.

Si eso no funciona para su caso de uso, prueba la funcion `v1.train.init_from_checkpoint`. Se necesita un argumento `task_map`, que especifica la asignacion de los nombres antiguos a los nuevos.

Nota: A diferencia de los checkpoints basados en objetos, que pueden [diferir la carga](checkpoints.ipynb # loading_mechanics), los checkpoints basados en nombres requieren que todas las variables se construyan cuando se llama a la funcion. Algunos modelos difieren las variables de construccion hasta que llame a `build` o ejecute el modelo en un lote de datos.

El [repositorio de TensorFlow Estimator](https://github.com/tensorflow/estimator/blob/master/tensorflow_estimator/python/estimator/tools/checkpoint_converter.py) incluye una [herramienta de conversión](#checkpoint_converter) para actualizar los checkpoints para estimadores prefabricados de TensorFlow 1.X a 2.0. Puede servir como un ejemplo de como construir una herramienta en un caso de uso similar.

### Compatibilidad de modelos guardados

No existen problemas de compatibilidad significativos para los modelos guardados.

* TensorFlow 1.x saved_models funciona en TensorFlow 2.0.
* TensorFlow 2.0 saved_models incluso carga el trabajo en TensorFlow 1.x si todas las operaciones son compatibles.

## Estimadores

### Entrenamiento con estimadores

Los estimadores son compatibles con TensorFlow 2.0.

Cuando usa estimadores, puede usar `input_fn()`, `tf.estimator.TrainSpec` y` tf.estimator.EvalSpec` de TensorFlow 1.x.

Aqui hay un ejemplo usando `input_fn` con train y evalua las especificaciones.

#### Creando las especificacviones input_fn y train/eval 

In [0]:

# Define los estimadores de input_fn
def input_fn():
  datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
  mnist_train, mnist_test = datasets['train'], datasets['test']

  BUFFER_SIZE = 10000
  BATCH_SIZE = 64

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label[..., tf.newaxis]

  train_data = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  return train_data.repeat()

# Define las especificaciones de entrenamiento y de eval
train_spec = tf.estimator.TrainSpec(input_fn=input_fn,
                                    max_steps=STEPS_PER_EPOCH * NUM_EPOCHS)
eval_spec = tf.estimator.EvalSpec(input_fn=input_fn,
                                  steps=STEPS_PER_EPOCH)


### Usando una definición de modelo Keras

Hay algunas diferencias en como construir sus estimadores en TensorFlow 2.0.

Recomendamos que definas tu modelo usando Keras, luego usa la utilidad `tf.keras.estimator.model_to_estimator` para convertir su modelo en un estimador. El siguiente codigo muestra como usar esta utilidad al crear y entrenar un estimador.

In [0]:
def make_model():
  return tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.02),
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [0]:
model = make_model()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

estimator = tf.keras.estimator.model_to_estimator(
  keras_model = model
)

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

### Usando un `model_fn` personalizado

Si tienes un estimador personalizado `model_fn` existente que necesita mantener, puedes convertir su `model_fn` para usar un modelo Keras.

Sin embargo, por razones de compatibilidad, un `model_fn` personalizado se ejecutara en modo grafico estilo 1.x. Esto significa que no hay una eager execution ni dependencias de control automatico.

<a name="minimal_changes"></a>

#### Model_fn personalizado con cambios minimos
Para que tu `model_fn` personalizado funcione en TF 2.0, si prefieres cambios minimos en el codigo existente, se pueden usar simbolos` tf.compat.v1` como `optimizers` y` metrics`.

Usar un modelo Keras en un `model_fn` personalizado es similar a usarlo en un ciclo de entrenamiento personalizado:

* Establece la fase de "training" de manera apropiada, segun el argumento de "mode".
* Pasa explicitamente las `variables_tranables` del modelo al optimizador.

Pero hay diferencias importantes, en relacion con un [bucle personalizado] (# custom_loop):

* En lugar de usar `Model.losses`, extrae las perdidas usando` Model.get_losses_for`.
* Extrae las actualizaciones del modelo usando `Model.get_updates_for`.

Nota: Las "Updates" son cambios que deben aplicarse a un modelo despues de cada lote. Por ejemplo, los promedios moviles de la media y la varianza en una capa `layers.BatchNormalization`.

El siguiente codigo crea un estimador a partir de un `model_fn` personalizado, que ilustra todas estas preocupaciones.

In [0]:
def my_model_fn(features, labels, mode):
  model = make_model()

  optimizer = tf.compat.v1.train.AdamOptimizer()
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

  training = (mode == tf.estimator.ModeKeys.TRAIN)
  predictions = model(features, training=training)

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  reg_losses = model.get_losses_for(None) + model.get_losses_for(features)
  total_loss = loss_fn(labels, predictions) + tf.math.add_n(reg_losses)

  accuracy = tf.compat.v1.metrics.accuracy(labels=labels,
                                           predictions=tf.math.argmax(predictions, axis=1),
                                           name='acc_op')

  update_ops = model.get_updates_for(None) + model.get_updates_for(features)
  minimize_op = optimizer.minimize(
      total_loss,
      var_list=model.trainable_variables,
      global_step=tf.compat.v1.train.get_or_create_global_step())
  train_op = tf.group(minimize_op, update_ops)

  return tf.estimator.EstimatorSpec(
    mode=mode,
    predictions=predictions,
    loss=total_loss,
    train_op=train_op, eval_metric_ops={'accuracy': accuracy})

# Creaa el estimador y el entrenador
estimator = tf.estimator.Estimator(model_fn=my_model_fn)
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

#### `model_fn` personalizado con símbolos TF 2.0
Si deseas deshacerse de todos los simbolos TF 1.x y actualizar tu `model_fn` personalizado a TF 2.0 nativo, debes actualizar el optimizador y las metricas a` tf.keras.optimizers` y `tf.keras.metrics`.

En el `model_fn` personalizado, ademas de los [cambios](#minimal_changes) anteriores , se deben realizar más actualizaciones:

* Utiliza [`tf.keras.optimizers`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/optimizers) en lugar de `v1.train.Optimizer`.
* Explicitamente pasa las `variables_tranables` del modelo a las` tf.keras.optimizers`.
* Para calcular el `train_op / minimizar_op`,
  * Usa `Optimizer.get_updates ()` si la perdida es perdida escalar `Tensor` (no es invocable). El primer elemento en la lista devuelta es el deseado `train_op / minimice_op`.
  * Si la perdida es invocable (como una funcion), usa `Optimizer.minimize()` para obtener `train_op / minimice_op`.
* Utiliza [`tf.keras.metrics`](https://www.tensorflow.org/api_docs/python/tf/keras/metrics) en lugar de` tf.compat.v1.metrics` para la evaluacion.

Para el ejemplo anterior de `my_model_fn`, el codigo migrado con símbolos 2.0 se muestra como:

In [0]:
def my_model_fn(features, labels, mode):
  model = make_model()

  training = (mode == tf.estimator.ModeKeys.TRAIN)
  loss_obj = tf.keras.losses.SparseCategoricalCrossentropy()
  predictions = model(features, training=training)

  # Obten ambas perdidas incondicionales (la parte None)
  # y las pérdidas condicionales de entrada (la parte de caracteristicas).
  reg_losses = model.get_losses_for(None) + model.get_losses_for(features)
  total_loss = loss_obj(labels, predictions) + tf.math.add_n(reg_losses)

  # Actualiza a tf.keras.metrics.
  accuracy_obj = tf.keras.metrics.Accuracy(name='acc_obj')
  accuracy = accuracy_obj.update_state(
      y_true=labels, y_pred=tf.math.argmax(predictions, axis=1))

  train_op = None
  if training:
    # Actualiza a tf.keras.optimizers.
    optimizer = tf.keras.optimizers.Adam()
    # Asigna manualmente la variable tf.compat.v1.global_step a 
    # optimizer.iterations
    # para hacer que tf.compat.v1.train.global_step aumente correctamente.
    # Esta asignacion es imprescindible para cualquier `tf.train.SessionRunHook`
    # especificado en estimador, ya que SessionRunHooks confía en el paso global.
    optimizer.iterations = tf.compat.v1.train.get_or_create_global_step()
    # Obten las actualizaciones incondicionales (la parte Ninguna)
    # y las actualizaciones condicionales de entrada (la parte de caracterIsticas).
    update_ops = model.get_updates_for(None) + model.get_updates_for(features)
    # Computa minimize_op.
    minimize_op = optimizer.get_updates(
        total_loss,
        model.trainable_variables)[0]
    train_op = tf.group(minimize_op, *update_ops)

  return tf.estimator.EstimatorSpec(
    mode=mode,
    predictions=predictions,
    loss=total_loss,
    train_op=train_op,
    eval_metric_ops={'Accuracy': accuracy_obj})

# Crea el Estimator y Train
estimator = tf.estimator.Estimator(model_fn=my_model_fn)
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

### Estimadores prefabricados

[Estimadores prefabricados](https://www.tensorflow.org/guide/premade_estimators) en la familia de `tf.estimator.DNN *`, `tf.estimator.Linear *` y `tf.estimator.DNNLinearCombined *` son todavia compatible con la API TensorFlow 2.0, sin embargo, algunos argumentos han cambiado:

1. `input_layer_partitioner`: eliminado en 2.0.
2. `loss_reduction`: actualizado a` tf.keras.losses.Reduction` en lugar de `tf.compat.v1.losses.Reduction`. Su valor predeterminado tambien se cambia a `tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE` de` tf.compat.v1.losses.Reduction.SUM`.
3. `optimizer`,` dnn_optimizer` y `linear_optimizer`: este argumento se ha actualizado a` tf.keras.optimizers` en lugar del `tf.compat.v1.train.Optimizer`.

Para migrar los cambios anteriores:
1. No se necesitas migracion para `input_layer_partitioner` ya que [` Estrategia de distribución`](https://www.tensorflow.org/guide/distribute_strategy) lo manejara automaticamente en TF 2.0.
2. Para `loss_reduction`, marca [` tf.keras.losses.Reduction`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/losses/Reduction) para ver opciones admitidas
3. Para los argumentos `optimizer`, si no pasa un argumento` optimizer`, `dnn_optimizer` o` linear_optimizer`, o si especifica el argumento `optimizer` como una` cadena` en su codigo, no necesitas cambiar nada. `tf.keras.optimizers` se usa por defecto. De lo contrario, debes actualizarlo desde `tf.compat.v1.train.Optimizer` a su correspondiente [` tf.keras.optimizers`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/optimizadores)

#### Convertidor de checkpoints

<a id="checkpoint_converter"></a>

La migración a `keras.optimizers` rompera los checkpoints guardados con TF 1.x, ya que`tf.keras.optimizers` genera un conjunto diferente de variables que se guardaran en los checkpoints. Para volver a utilizar el antiguo checkpoints despues de su migración a TF 2.0, prueba la [herramienta de conversión de punto de control](https://github.com/tensorflow/estimator/blob/master/tensorflow_estimator/python/estimator/tools/checkpoint_converter.py).





In [0]:
! curl -O https://raw.githubusercontent.com/tensorflow/estimator/master/tensorflow_estimator/python/estimator/tools/checkpoint_converter.py

La herramienta contiene ayuda integrada:

In [0]:
! python checkpoint_converter.py -h

<a id="tensorshape"></a>

## TensorShape

Esta clase se simplifico para contener objetos `int`s, en lugar de objetos` tf.compat.v1.Dimension`. Por lo tanto, no es necesario llamar a `.value()` para obtener un `int`.

Los objetos individuales `tf.compat.v1.Dimension` todavia son accesibles desde` tf.TensorShape.dims`.


A continuación se muestran las diferencias entre TensorFlow 1.xy TensorFlow 2.0.

In [0]:

# Crea una forma y elige un indice
i = 0
shape = tf.TensorShape([16, None, 256])
shape

Si tienes esto en TF 1.x:

```python
value = shape[i].value
```

Haz esto en TF 2.0:


In [0]:
value = shape[i]
value

Si tienes esto en TF 1.x:

```python
for dim in shape:
    value = dim.value
    print(value)
```

Haz esto en TF 2.0:


In [0]:
for value in shape:
  print(value)


Si tienes esto en TF 1.x (o usas cualquier otro metodo de dimensión):

```python
dim = shape[i]
dim.assert_is_compatible_with(other_dim)
```

Haz esto en TF 2.0:

In [0]:
other_dim = 16
Dimension = tf.compat.v1.Dimension

if shape.rank is None:
  dim = Dimension(None)
else:
  dim = shape.dims[i]
dim.is_compatible_with(other_dim) # o cualquier otro metodo de dimension

In [0]:
shape = tf.TensorShape(None)

if shape:
  dim = shape.dims[i]
  dim.is_compatible_with(other_dim) # o cualquier otro metodo de dimension


El valor booleano de un `tf.TensorShape` es `True` si se conoce el rango, de otro modo es `False`. 

In [0]:
print(bool(tf.TensorShape([])))      # Escalar
print(bool(tf.TensorShape([0])))     # Valor de longitud 0
print(bool(tf.TensorShape([1])))     # Vector de longitud 1
print(bool(tf.TensorShape([None])))  # Vector de longitud desconocida
print(bool(tf.TensorShape([1, 10, 100])))       # Tensor 3D
print(bool(tf.TensorShape([None, None, None]))) # Tensor 3D con dimenciones no conocidas
print()
print(bool(tf.TensorShape(None)))  # Un tensor con rango desconocido.

## Otros cambios

* Elimina `tf.colocate_with`: los algoritmos de colocacion de dispositivos de TensorFlow han mejorado significativamente. Esto ya no deberia ser necesario. Si eliminarlo provoca una degradación del rendimiento [reporta un bug](https://github.com/tensorflow/tensorflow/issues).

## Conclusiones

El proceso general es:

1. Ejecuta el script de actualización.
2. Elimina los símbolos contrib.
3. Cambia sus modelos a un estilo orientado a objetos (Keras).
4. Usa los bucles de entrenamiento y evaluacion `tf.keras` o` tf.estimator` donde puedas.
5. De lo contrario, usa ciclos personalizados, pero asegurate de evitar sesiones y colecciones.


Se necesita un poco de trabajo para convertir el codigo a TensorFlow 2.0 idiomatico, pero cada cambio resulta en:

* Menos lineas de codigo.
* Mayor claridad y simplicidad.
* Depuracion mas facil.


